# Chat Agent Executor

In this example we will build a chat executor that uses function calling from
scratch.

## Setup¶

First we need to install the packages required

```bash
yarn add langchain @langchain/openai @langchain/langgraph
```

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the
search tool we will use)

```bash
export OPENAI_API_KEY=
export TAVILY_API_KEY=
```

Optionally, we can set API key for
[LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=
```

## Set up the tools

We will first define the tools we want to use. For this simple example, we will
use a built-in search tool via Tavily. However, it is really easy to create your
own tools - see documentation
[here](https://js.langchain.com/docs/modules/agents/tools/dynamic) on how to do
that.

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const tools = [new TavilySearchResults({ maxResults: 1 })];

We can now wrap these tools in a simple ToolExecutor. This is a real simple
class that takes in a ToolInvocation and calls that tool, returning the output.

A ToolInvocation is any type with `tool` and `toolInput` attribute.

In [3]:
import { ToolExecutor } from "@langchain/langgraph/prebuilt";

const toolExecutor = new ToolExecutor({
  tools,
});

## Set up the model

Now we need to load the chat model we want to use. Importantly, this should
satisfy two criteria:

1. It should work with messages. We will represent all agent state in the form
   of messages, so it needs to be able to work well with them.
2. It should work with OpenAI function calling. This means it should either be
   an OpenAI model or a model that exposes a similar interface.

Note: these model requirements are not requirements for using LangGraph - they
are just requirements for this one example.

In [4]:
import { ChatOpenAI } from "@langchain/openai";

// We will set streaming=True so that we can stream tokens
// See the streaming section for more information on this.
const model = new ChatOpenAI({
  temperature: 0,
  streaming: true,
});

After we've done this, we should make sure the model knows that it has these
tools available to call. We can do this by converting the LangChain tools into
the format for OpenAI function calling, and then bind them to the model class.

In [5]:
import { convertToOpenAIFunction } from "@langchain/core/utils/function_calling";

const toolsAsOpenAIFunctions = tools.map((tool) =>
  convertToOpenAIFunction(tool)
);
const newModel = model.bind({
  functions: toolsAsOpenAIFunctions,
});

### Define the agent state

The main type of graph in `langgraph` is the `StatefulGraph`. This graph is
parameterized by a state object that it passes around to each node. Each node
then returns operations to update that state. These operations can either SET
specific attributes on the state (e.g. overwrite the existing values) or ADD to
the existing attribute. Whether to set or add is denoted by annotating the state
object you construct the graph with.

For this example, the state we will track will just be a list of messages. We
want each node to just add messages to that list. To do this, we define our state via the `Annotation` function, with a single attribute `messages` that is a list of `BaseMessage`s. The value of our attribute `messages` is another `Annotation` which has two sub-attributes: `reducer` and `default`. The `reducer` key must be a factory that returns a function that takes the current value of the attribute and the new value to add, and returns the new value of the attribute.

The `default` key must be a factory that returns the default value for that
attribute (this key is optional).

In [6]:
import { BaseMessage } from "@langchain/core/messages";
import { Annotation } from "@langchain/langgraph";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
    default: () => [],
  })
})

TypeError: Cannot read properties of undefined (reading 'Root')

## Define the nodes

We now need to define a few different nodes in our graph. In `langgraph`, a node
can be either a function or a
[runnable](https://js.langchain.com/docs/expression_language/). There are two
main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node
   will then execute that action.

We will also need to define some edges. Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of
several paths may be taken. The path that is taken is not known until that node
is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either: a. If the
   agent said to take an action, then the function to invoke tools should be
   called b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the
   agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional
edge to take.

In [ ]:
import { FunctionMessage } from "@langchain/core/messages";
import { AgentAction } from "@langchain/core/agents";

// Define the function that determines whether to continue or not
const shouldContinue = (state: typeof AgentState.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1];
  // If there is no function call, then we finish
  if (
    !("function_call" in lastMessage.additional_kwargs) ||
    !lastMessage.additional_kwargs.function_call
  ) {
    return "end";
  }
  // Otherwise if there is, we continue
  return "continue";
};

// Define the function to execute tools
const _getAction = (state: typeof AgentState.State): AgentAction => {
  const { messages } = state;
  // Based on the continue condition
  // we know the last message involves a function call
  const lastMessage = messages[messages.length - 1];
  if (!lastMessage) {
    throw new Error("No messages found.");
  }
  if (!lastMessage.additional_kwargs.function_call) {
    throw new Error("No function call found in message.");
  }
  // We construct an AgentAction from the function_call
  return {
    tool: lastMessage.additional_kwargs.function_call.name,
    toolInput: JSON.stringify(
      lastMessage.additional_kwargs.function_call.arguments,
    ),
    log: "",
  };
};

// Define the function that calls the model
const callModel = async (state: typeof AgentState.State) => {
  const { messages } = state;
  const response = await newModel.invoke(messages);
  // We return a list, because this will get added to the existing list
  return {
    messages: [response],
  };
};

const callTool = async (state: typeof AgentState.State) => {
  const action = _getAction(state);
  // We call the tool_executor and get back a response
  const response = await toolExecutor.invoke(action);
  // We use the response to create a FunctionMessage
  const functionMessage = new FunctionMessage({
    content: response,
    name: action.tool,
  });
  // We return a list, because this will get added to the existing list
  return { messages: [functionMessage] };
};

## Define the graph

We can now put it all together and define the graph!

In [ ]:
import { END, START, StateGraph } from "@langchain/langgraph";

// Define a new graph
const workflow = new StateGraph(AgentState);

// Define the two nodes we will cycle between
workflow.addNode("agent", callModel);
workflow.addNode("action", callTool);

// Set the entrypoint as `agent`
// This means that this node is the first one called
workflow.addEdge(START, "agent");

// We now add a conditional edge
workflow.addConditionalEdges(
  // First, we define the start node. We use `agent`.
  // This means these are the edges taken after the `agent` node is called.
  "agent",
  // Next, we pass in the function that will determine which node is called next.
  shouldContinue,
  // Finally we pass in a mapping.
  // The keys are strings, and the values are other nodes.
  // END is a special node marking that the graph should finish.
  // What will happen is we will call `should_continue`, and then the output of that
  // will be matched against the keys in this mapping.
  // Based on which one it matches, that node will then be called.
  {
    // If `tools`, then we call the tool node.
    continue: "action",
    // Otherwise we finish.
    end: END,
  },
);

// We now add a normal edge from `tools` to `agent`.
// This means that after `tools` is called, `agent` node is called next.
workflow.addEdge("action", "agent");

// Finally, we compile it!
// This compiles it into a LangChain Runnable,
// meaning you can use it as you would any other runnable
const app = workflow.compile();

## Use it!

We can now use it! This now exposes the
[same interface](https://python.langchain.com/docs/expression_language/) as all
other LangChain runnables.

In [ ]:
import { HumanMessage } from "@langchain/core/messages";

const inputs = {
  messages: [new HumanMessage("what is the weather in sf")],
};
await app.invoke(inputs);

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: { content: "what is the weather in sf", additional_kwargs: {} },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "what is the weather in sf",
      name: undefined,
      additional_kwargs: {}
    },
    AIMessageChunk {
      lc_serializable: true,
      lc_kwargs: { content: "", additional_kwargs: { function_call: [Object] } },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "",
      name: undefined,
      additional_kwargs: {
        function_call: {
          name: "tavily_search_results_json",
          arguments: '{"input":"weather in San Francisco"}'
        }
      }
    },
    FunctionMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "[]",
        name: "tavily_search_results_json",
        additional_kwargs: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "[]",
      name: "tavily_search_results_j

This may take a little bit - it's making a few calls behind the scenes. In order
to start seeing some intermediate results as they happen, we can use streaming.
See below for more information on that.

## Streaming

LangGraph has support for several different types of streaming.

### Streaming Node Output

One of the benefits of using LangGraph is that it is easy to stream output as
it's produced by each node.

In [ ]:
const inputs = {
  messages: [new HumanMessage("what is the weather in sf")],
};
for await (const output of await app.stream(inputs)) {
  console.log("output", output);
  console.log("-----\n");
}

output {
  agent: {
    messages: [
      AIMessageChunk {
        lc_serializable: true,
        lc_kwargs: { content: "", additional_kwargs: [Object] },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "",
        name: undefined,
        additional_kwargs: { function_call: [Object] }
      }
    ]
  }
}
-----

output {
  action: {
    messages: [
      FunctionMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "[]",
          name: "tavily_search_results_json",
          additional_kwargs: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "[]",
        name: "tavily_search_results_json",
        additional_kwargs: {}
      }
    ]
  }
}
-----

output {
  agent: {
    messages: [
      AIMessageChunk {
        lc_serializable: true,
        lc_kwargs: {
          content: "I couldn't find the current weather in San Francisco. You may want to check a reliable weather websi"... 46 more characters